In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Part 1: Data Processing and Feature Engineering (Day 1)
# 1. Data Cleaning

# Load the dataset using Pandas.

In [2]:
train = pd.read_csv(r'C:\Users\Basavaraj Humanabad\Documents\train.csv')
test = pd.read_csv(r'C:\Users\Basavaraj Humanabad\Documents\test.csv')
stores = pd.read_csv(r'C:\Users\Basavaraj Humanabad\Documents\stores.csv')
oil = pd.read_csv(r'C:\Users\Basavaraj Humanabad\Documents\oil.csv')
holidays = pd.read_csv(r'C:\Users\Basavaraj Humanabad\Documents\holidays_events.csv')

# Convert date columns to datetime format

In [3]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

# Convert date columns to datetime format

In [4]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

# Fill missing values in oil prices using interpolation

In [5]:
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate()
oil['dcoilwtico'] 

0         NaN
1       93.14
2       92.97
3       93.12
4       93.20
        ...  
1213    47.65
1214    46.40
1215    46.46
1216    45.96
1217    47.26
Name: dcoilwtico, Length: 1218, dtype: float64

# Merge data

In [6]:
train = train.merge(stores, on='store_nbr', how='left')
train = train.merge(oil, on='date', how='left')
train = train.merge(holidays, on='date', how='left')

test = test.merge(stores, on='store_nbr', how='left')
test = test.merge(oil, on='date', how='left')
test = test.merge(holidays, on='date', how='left')

# 2.Feature Engineering

In [7]:
train['day'] = train['date'].dt.day
train['week'] = train['date'].dt.isocalendar().week
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['day_of_week'] = train['date'].dt.dayofweek
train['is_weekend'] = train['day_of_week'].isin([5,6]).astype(int)
train['is_december'] = (train['month'] == 12).astype(int)

# Identify seasonal trends (e.g., are sales higher in December?).

In [8]:
train["is_december"] = (train["month"] == 12).astype(int)
train["is_december"]

0          0
1          0
2          0
3          0
4          0
          ..
3054343    0
3054344    0
3054345    0
3054346    0
3054347    0
Name: is_december, Length: 3054348, dtype: int32

# Event-based Features:Create binary flags for holidays, promotions, and economic events.

In [9]:
train["is_holiday"] = train["type_x"].notna().astype(int)
train["is_holiday"]

0          1
1          1
2          1
3          1
4          1
          ..
3054343    1
3054344    1
3054345    1
3054346    1
3054347    1
Name: is_holiday, Length: 3054348, dtype: int32

# Identify if a day is a government payday (15th and last day of the month).

In [10]:
train["is_govt_payday"] = train["day"].isin([15] + train["date"].dt.days_in_month.tolist()).astype(int)
train["is_govt_payday"]

0          0
1          0
2          0
3          0
4          0
          ..
3054343    1
3054344    1
3054345    1
3054346    1
3054347    1
Name: is_govt_payday, Length: 3054348, dtype: int32

# Consider earthquake impact (April 16, 2016) as a separate feature.

In [11]:
train["earthquake_impact"] = (train["date"] == "2016-04-16").astype(int)
train["earthquake_impact"]

0          0
1          0
2          0
3          0
4          0
          ..
3054343    0
3054344    0
3054345    0
3054346    0
3054347    0
Name: earthquake_impact, Length: 3054348, dtype: int32

# Rolling Statistics

In [12]:
train['sales_lag_7'] = train.groupby(['store_nbr', 'family'])['sales'].shift(7)
train['sales_lag_30'] = train.groupby(['store_nbr', 'family'])['sales'].shift(30)
train['rolling_mean_7'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7).mean())
train['rolling_std_7'] = train.groupby(['store_nbr', 'family'])['sales'].transform(lambda x: x.rolling(window=7).std())

# Store-Specific Aggregations

In [ ]:
store_sales_avg = train.groupby('store_nbr')['sales'].mean().rename('store_avg_sales')
train = train.merge(store_sales_avg, on='store_nbr', how='left')
train

# 3.Exploratory Data Analysis (EDA)

In [ ]:
plt.figure(figsize=(12,6))
sns.lineplot(data=train, x='date', y='sales', ci=None)
plt.title('Sales Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

# Checking correlation between oil prices and sales

In [ ]:
plt.figure(figsize=(12,6))
sns.scatterplot(data=train, x='dcoilwtico', y='sales')
plt.title('Correlation between Oil Prices and Sales')
plt.xlabel('Oil Price')
plt.ylabel('Sales')
plt.show()

# Checking anomalies

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data=train, y='sales')
plt.title('Sales Distribution and Anomalies')
plt.show()

# Documenting preprocessing steps

In [ ]:
with open('preprocessing_documentation.txt', 'w') as f:
    f.write("Data Processing and Feature Engineering Steps:\n")
    f.write("1. Handled missing values in oil prices using interpolation.\n")
    f.write("2. Converted date columns to proper datetime format.\n")
    f.write("3. Merged stores, oil, and holidays data with sales data.\n")
    f.write("4. Extracted time-based features: day, week, month, year, day_of_week.\n")
    f.write("5. Created binary indicators for holidays, paydays, and economic events.\n")
    f.write("6. Computed rolling sales statistics and lagged features.\n")
    f.write("7. Performed store-specific aggregations for average sales.\n")
    f.write("8. Conducted EDA with time series visualization and anomaly detection.\n")

# Part 2: Model Selection, Forecasting, and Evaluation (Day 2)
# 1. Model Training

In [ ]:
train.head(2)

In [ ]:
from sklearn.model_selection import train_test_split  # Import train_test_split

# Define feature set and target variable
X = train[['day', 'week', 'month', 'year', 'day_of_week', 'is_weekend', 'is_december', 
           'is_holiday', 'is_govt_payday', 'earthquake_impact', 'sales_lag_7', 
           'sales_lag_30', 'rolling_mean_7', 'rolling_std_7', 'store_avg_sales']]
y = train['sales']

# Perform train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Import necessary library
from sklearn.metrics import mean_absolute_error

# Ensure that y_pred_baseline is properly defined
y_pred_baseline = X_val['sales_lag_7'].fillna(y_train.mean())  # Fill NaNs with training mean

# Compute Mean Absolute Error (MAE)
mae_baseline = mean_absolute_error(y_val, y_pred_baseline)

print("Baseline Model MAE:", mae_baseline)